In [1]:
import sys
import os

# This adds the main project folder ("QUANT-DEV-UNVESTING") to the list of places
# Python looks for code. It goes one level up from the current working directory.
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Purpose
This testing file will be used to test some of the indicators

In [4]:
import yfinance as yf
import pandas as pd

start_date = '2024-06-15'
end_date = '2025-06-15'
tickers = ['EC', 'VOO', 'NU', 'NVDA', 'AAPL']
df = yf.download(tickers, start_date, end_date)['Close']
df

[*********************100%***********************]  5 of 5 completed


Ticker,AAPL,EC,NU,NVDA,VOO
Date,,,,,
2024-06-17,215.661423,11.073669,11.83,130.938644,496.288025
2024-06-18,213.292480,11.289147,11.69,135.537186,497.610291
2024-06-20,208.703964,11.345359,12.08,130.738708,496.228790
2024-06-21,206.524155,11.242305,12.25,126.530022,495.143341
2024-06-24,207.171127,11.870000,11.86,118.072701,493.811218
...,...,...,...,...,...
2025-06-09,201.449997,8.930000,11.92,142.620102,551.250000
2025-06-10,202.669998,9.020000,11.97,143.950012,554.390015
2025-06-11,198.779999,9.310000,12.76,142.830002,552.859985


# Sharpe Ratio

In [5]:
from indicators import SharpeRatio
rfr = 0.03
periods_per_year = 252
for ticker in tickers:
    prices = df[ticker]
    sr = SharpeRatio(prices, rfr, periods_per_year).calculate()
    if sr > 0:
        msg = 'Performance over risk-free rate. ACCEPT.'
    else:
        msg = 'Performance below risk-free rate. REJECT.' 
    print(f'Sharpe for: {ticker} is {sr}. {msg}')
    

Sharpe for: EC is -0.23930325612902442. Performance below risk-free rate. REJECT.
Sharpe for: VOO is 0.4667682531060147. Performance over risk-free rate. ACCEPT.
Sharpe for: NU is 0.200254437250595. Performance over risk-free rate. ACCEPT.
Sharpe for: NVDA is 0.3827488480038188. Performance over risk-free rate. ACCEPT.
Sharpe for: AAPL is -0.22475351811167427. Performance below risk-free rate. REJECT.
